In [1]:
import openai
import logging
from pydantic import BaseModel, ValidationError
from typing import List, Literal

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="sk-")
openai.api_key = "sk-"

In [3]:
# Pydantic models
class LinkedPremiseInput(BaseModel):
    premise_text: str
    linked_claim_text: str

class StanceAnnotatedPremise(BaseModel):
    premise_text: str
    linked_claim_text: str
    stance: Literal["pro", "con"]

In [5]:
# Main function
def classify_stance_with_openai(linked_premises: List[LinkedPremiseInput]) -> List[StanceAnnotatedPremise]:
    system_prompt = """
You are an expert in argument mining. Your task is to determine whether a premise supports (pro) or opposes (con) the linked claim.
Respond with one word: "pro" or "con".
"""
    results = []

    for item in linked_premises:
        premise = item.premise_text
        claim = item.linked_claim_text

        user_prompt = f"""Claim: "{claim}"\nPremise: "{premise}"\nWhat is the stance of the premise with respect to the claim?"""

        response = client.chat.completions.create(
            model="gpt-4.1",
            messages=[
                {"role": "system", "content": system_prompt.strip()},
                {"role": "user", "content": user_prompt.strip()}
            ],
            temperature=0.0,
        )

        stance_raw = response.choices[0].message.content.strip().lower()
        stance = stance_raw if stance_raw in ("pro", "con") else "con"  # default fallback

        try:
            validated = StanceAnnotatedPremise(
                premise_text=premise,
                linked_claim_text=claim,
                stance=stance
            )
            results.append(validated)
        except ValidationError as e:
            print("❌ Failed to parse or validate OpenAI output.")
            print("Raw Output:\n", stance_raw)
            raise e

    return results

In [6]:
from uuid import uuid4

linked_premises = [
    LinkedPremiseInput(
        premise_text="People working remotely often report fewer distractions and better focus.",
        linked_claim_text="Remote work improves employee productivity."
    ),
    LinkedPremiseInput(
        premise_text="Most EVs are charged with electricity generated from fossil fuels.",
        linked_claim_text="EVs have zero tailpipe emissions."
    )
]

results = classify_stance_with_openai(linked_premises)
for r in results:
    print(r.model_dump())


{'premise_text': 'People working remotely often report fewer distractions and better focus.', 'linked_claim_text': 'Remote work improves employee productivity.', 'stance': 'pro'}
{'premise_text': 'Most EVs are charged with electricity generated from fossil fuels.', 'linked_claim_text': 'EVs have zero tailpipe emissions.', 'stance': 'con'}


In [8]:
from google.colab import files
import json

# Save the linked output
with open("output.json", "w") as f:
    json.dump([r.model_dump() for r in results], f, indent=2)

# Download to local machine
files.download("output.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>